In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

df= pd.read_csv('C://Users//User//Desktop//MSc Westminster//Dissertation//DataSets//Heart_Disease_Indicators.csv')

In [2]:
from sklearn.model_selection import train_test_split

target_size = 50000

num_class_0 = int(target_size * df['HeartDiseaseorAttack'].value_counts(normalize=True)[0])
num_class_1 = target_size - num_class_0

df_class_0 = df[df['HeartDiseaseorAttack'] == 0]
df_class_1 = df[df['HeartDiseaseorAttack'] == 1]

df_class_0_sampled = df_class_0.sample(n=num_class_0, random_state=15)
df_class_1_sampled = df_class_1.sample(n=num_class_1, random_state=15)

df_sampled = pd.concat([df_class_0_sampled, df_class_1_sampled])

df_sampled = df_sampled.sample(frac=1, random_state=15).reset_index(drop=True)
df= df_sampled
print(df['HeartDiseaseorAttack'].value_counts(normalize=True))

0    0.90582
1    0.09418
Name: HeartDiseaseorAttack, dtype: float64


In [3]:
df.drop_duplicates(inplace= True)
df.shape

(48050, 22)

In [4]:
df= df[df['Diabetes'] != 1].copy()
df.loc[df['Diabetes'] == 2, 'Diabetes'] = 1
print(df['Diabetes'].value_counts())

0    40175
1     6968
Name: Diabetes, dtype: int64


In [5]:
categorical_columns= ['HighBP', 'HighChol', 'CholCheck', 'Smoker', 'Stroke', 'Diabetes', 'PhysActivity',
                      'Fruits', 'Veggies', 'HvyAlcoholConsump', 'AnyHealthcare', 'NoDocbcCost', 'DiffWalk',
                      'Sex', 'Age', 'Education', 'Income'
                     ]
df[categorical_columns]= df[categorical_columns].astype(str)
df= pd.get_dummies(df, columns= categorical_columns, drop_first= True)

In [6]:
X= df.drop(['HeartDiseaseorAttack'], axis= 1)
y= df['HeartDiseaseorAttack']

In [7]:
def fisher_score(X, y):
    labels= np.unique(y)
    mean_overall= np.mean(X, axis=0)
    fisher_scores= []
    for i in range(X.shape[1]):
        numerator= 0
        denominator= 0
        for label in labels:
            X_label= X[y== label]
            mean_label= np.mean(X_label[:, i])
            numerator += len(X_label) * (mean_label - mean_overall[i])**2
            denominator += np.sum((X_label[:, i] - mean_label)**2)
        fisher_scores.append(numerator / (denominator + 1e-10))
    return np.array(fisher_scores)

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size= 0.2, random_state= 15, stratify= y)

continuous_columns = ['BMI', 'GenHlth', 'MentHlth', 'PhysHlth']

from sklearn.preprocessing import StandardScaler
ss= StandardScaler()
X_train= ss.fit_transform(X_train)
X_test= ss.transform(X_test)

In [9]:
from imblearn.combine import SMOTETomek

smt= SMOTETomek(random_state= 15)
X_sm_tl, y_sm_tl= smt.fit_resample(X_train, y_train)

In [10]:
fisher_scores= fisher_score(X_sm_tl, y_sm_tl)
k= 10
top_k_indices= np.argsort(fisher_scores)[-k:]

X_sm_tl_selected= X_sm_tl[:, top_k_indices]
X_sm_tl_test_selected= X_test[:, top_k_indices]

In [11]:
feature_names = X.columns
fisher_df= pd.DataFrame({'Feature': feature_names,
                          'Fisher Score': fisher_scores})

fisher_df= fisher_df.sort_values(by= 'Fisher Score', ascending= False)
fisher_df

,Feature,Fisher Score
1,GenHlth,1.878361e-01
4,HighBP_1,1.654801e-01
5,HighChol_1,1.194327e-01
16,DiffWalk_1,8.262060e-02
3,PhysHlth,6.377530e-02
8,Stroke_1,5.834158e-02
9,Diabetes_1,5.515923e-02
7,Smoker_1,4.430414e-02
21,Age_13,3.403652e-02
41,Income_8,2.991167e-02


--- LogisticRegression ---

In [12]:
from sklearn.metrics import confusion_matrix, classification_report, precision_score, roc_auc_score, accuracy_score
from sklearn.linear_model import LogisticRegression
lr= LogisticRegression(random_state= 15)
lr.fit(X_sm_tl_selected, y_sm_tl)
y_pred_lr= lr.predict(X_sm_tl_test_selected)
y_pred_prob_lr= lr.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_lr, labels= lr.classes_)
print(classification_report(y_test, y_pred_lr, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_lr, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_lr))
print('AUC:', roc_auc_score(y_test, y_pred_prob_lr))

              precision    recall  f1-score   support

           0       0.97      0.74      0.84      8517
           1       0.25      0.78      0.37       912

    accuracy                           0.75      9429
   macro avg       0.61      0.76      0.61      9429
weighted avg       0.90      0.75      0.80      9429

Precision: 0.24601524601524602
Accuracy: 0.7477993424541308
AUC: 0.8306252561955553


--- DecisionTreeClassifier ---

In [13]:
from sklearn.tree import DecisionTreeClassifier
dt= DecisionTreeClassifier(random_state=15)
dt.fit(X_sm_tl_selected, y_sm_tl)
y_pred_dt= dt.predict(X_sm_tl_test_selected)
y_pred_prob_dt= dt.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_dt, labels= dt.classes_)
print(classification_report(y_test, y_pred_dt, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_dt, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_dt))
print('AUC:', roc_auc_score(y_test, y_pred_prob_dt))

              precision    recall  f1-score   support

           0       0.92      0.97      0.94      8517
           1       0.36      0.17      0.23       912

    accuracy                           0.89      9429
   macro avg       0.64      0.57      0.58      9429
weighted avg       0.86      0.89      0.87      9429

Precision: 0.35664335664335667
Accuracy: 0.8902322621699014
AUC: 0.6822990371166852


--- Tuned - DecisionTreeClassifier ---

In [14]:
from sklearn.model_selection import GridSearchCV
param_grid= {
             'max_depth': [None, 10, 20, 30, 40, 50],
             'min_samples_split': [2, 10, 20],
             'min_samples_leaf': [1, 5, 10],
             'max_features': [None, 'sqrt', 'log2'],
             'criterion': ['gini', 'entropy']
            }

gs_dt= GridSearchCV(estimator= dt, param_grid= param_grid, cv= 5, scoring= 'precision')
gs_dt.fit(X_sm_tl_selected, y_sm_tl)

print("Best Parameters:", gs_dt.best_params_)
print("Best Precision Score:", gs_dt.best_score_)

Best Parameters: {'criterion': 'gini', 'max_depth': None, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 2}
Best Precision Score: 0.9627338839156911


In [15]:
tuned_dt= gs_dt.best_estimator_
y_pred_tuned_dt= tuned_dt.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_dt= tuned_dt.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_dt, labels= tuned_dt.classes_)
print(classification_report(y_test, y_pred_tuned_dt, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_dt, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_dt))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_dt))

              precision    recall  f1-score   support

           0       0.92      0.97      0.94      8517
           1       0.36      0.17      0.23       912

    accuracy                           0.89      9429
   macro avg       0.64      0.57      0.58      9429
weighted avg       0.86      0.89      0.87      9429

Precision: 0.35664335664335667
Accuracy: 0.8902322621699014
AUC: 0.6822990371166852


--- KNeighborsClassifier ---

In [16]:
from sklearn.neighbors import KNeighborsClassifier
knn= KNeighborsClassifier()
knn.fit(X_sm_tl_selected, y_sm_tl)
y_pred_knn= knn.predict(X_sm_tl_test_selected)
y_pred_prob_knn= knn.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_knn, labels= knn.classes_)
print(classification_report(y_test, y_pred_knn, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_knn, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_knn))
print('AUC:', roc_auc_score(y_test, y_pred_prob_knn))

              precision    recall  f1-score   support

           0       0.92      0.94      0.93      8517
           1       0.33      0.27      0.29       912

    accuracy                           0.88      9429
   macro avg       0.63      0.60      0.61      9429
weighted avg       0.87      0.88      0.87      9429

Precision: 0.32837837837837835
Accuracy: 0.8763389542899566
AUC: 0.712551998685807


--- Tuned - KNeighborsClassifier ---

In [17]:
param_grid= {
             'n_neighbors': np.arange(1,40),
             'weights': ['uniform', 'distance'],
             'metric': ['euclidean', 'manhattan', 'minkowski']
            }

gs_knn= GridSearchCV(estimator= knn, param_grid= param_grid, cv=5, scoring= 'precision')
gs_knn.fit(X_sm_tl_selected, y_sm_tl)
print("Best Parameters:", gs_knn.best_params_)
print("Best Precision Score:", gs_knn.best_score_)

Best Parameters: {'metric': 'manhattan', 'n_neighbors': 2, 'weights': 'uniform'}
Best Precision Score: 0.957244589798786


In [18]:
tuned_knn= gs_knn.best_estimator_
y_pred_tuned_knn= tuned_knn.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_knn= tuned_knn.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_knn, labels= tuned_knn.classes_)
print(classification_report(y_test, y_pred_tuned_knn, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_knn, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_knn))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_knn))

              precision    recall  f1-score   support

           0       0.91      0.97      0.94      8517
           1       0.29      0.13      0.18       912

    accuracy                           0.88      9429
   macro avg       0.60      0.55      0.56      9429
weighted avg       0.85      0.88      0.86      9429

Precision: 0.28641975308641976
Accuracy: 0.8849294729027468
AUC: 0.6337340798279602


--- GaussianNB ---

In [19]:
from sklearn.naive_bayes import GaussianNB
nb= GaussianNB()
nb.fit(X_sm_tl_selected, y_sm_tl)
y_pred_nb= nb.predict(X_sm_tl_test_selected)
y_pred_prob_nb= nb.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_nb, labels= nb.classes_)
print(classification_report(y_test, y_pred_nb, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_nb, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_nb))
print('AUC:', roc_auc_score(y_test, y_pred_prob_nb))

              precision    recall  f1-score   support

           0       0.96      0.80      0.87      8517
           1       0.27      0.68      0.38       912

    accuracy                           0.79      9429
   macro avg       0.61      0.74      0.63      9429
weighted avg       0.89      0.79      0.82      9429

Precision: 0.26715265866209265
Accuracy: 0.7881005408845052
AUC: 0.8209059177825978


--- SVM ---

In [20]:
from sklearn.svm import SVC
svc= SVC(kernel= 'rbf',probability= True, gamma= 1, random_state=15)
svc.fit(X_sm_tl_selected, y_sm_tl)
y_pred_svc= svc.predict(X_sm_tl_test_selected)
y_pred_prob_svc= svc.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_svc, labels= svc.classes_)
print(classification_report(y_test, y_pred_svc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_svc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_svc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_svc))

              precision    recall  f1-score   support

           0       0.94      0.86      0.90      8517
           1       0.29      0.53      0.38       912

    accuracy                           0.83      9429
   macro avg       0.62      0.70      0.64      9429
weighted avg       0.88      0.83      0.85      9429

Precision: 0.29191797346200243
Accuracy: 0.830098631880369
AUC: 0.7306844644045243


--- Random Forest ---

In [21]:
from sklearn.ensemble import RandomForestClassifier
rf= RandomForestClassifier(random_state=15)
rf.fit(X_sm_tl_selected, y_sm_tl)
y_pred_rf= rf.predict(X_sm_tl_test_selected)
y_pred_prob_rf= rf.predict_proba(X_sm_tl_test_selected)[:,1]

In [22]:
cm= confusion_matrix(y_test, y_pred_rf, labels= rf.classes_)
print(classification_report(y_test, y_pred_rf, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_rf, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_rf))
print('AUC:', roc_auc_score(y_test, y_pred_prob_rf))

              precision    recall  f1-score   support

           0       0.92      0.96      0.94      8517
           1       0.36      0.19      0.25       912

    accuracy                           0.89      9429
   macro avg       0.64      0.58      0.59      9429
weighted avg       0.86      0.89      0.87      9429

Precision: 0.36099585062240663
Accuracy: 0.8890656485311274
AUC: 0.7481939500771421


In [23]:
param_grid= {
             'n_estimators': [100, 200, 300],
             'max_depth': [None, 10, 20, 30],
             'min_samples_split': [2, 5, 10],
             'min_samples_leaf': [1, 2, 4],
            }

gs_tuned_rf= GridSearchCV(estimator= rf, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_tuned_rf.fit(X_sm_tl_selected, y_sm_tl)
print("Best Parameters:", gs_tuned_rf.best_params_)
print("Best Precision Score:", gs_tuned_rf.best_score_)

Fitting 5 folds for each of 108 candidates, totalling 540 fits
Best Parameters: {'max_depth': 30, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 300}
Best Precision Score: 0.9614154044672301


In [24]:
tuned_rf= gs_tuned_rf.best_estimator_
y_pred_tuned_rf= tuned_rf.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_rf= tuned_rf.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_rf, labels= tuned_rf.classes_)
print(classification_report(y_test, y_pred_tuned_rf, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_rf, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_rf))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_rf))

              precision    recall  f1-score   support

           0       0.92      0.97      0.94      8517
           1       0.38      0.20      0.26       912

    accuracy                           0.89      9429
   macro avg       0.65      0.58      0.60      9429
weighted avg       0.87      0.89      0.88      9429

Precision: 0.3818565400843882
Accuracy: 0.8913988758086754
AUC: 0.7883933500388285


--- AdaBoost ---

In [25]:
from sklearn.ensemble import AdaBoostClassifier
ada= AdaBoostClassifier(random_state=15)
ada.fit(X_sm_tl_selected, y_sm_tl)
y_pred_ada= ada.predict(X_sm_tl_test_selected)
y_pred_prob_ada= ada.predict_proba(X_sm_tl_test_selected)[:,1]

In [26]:
cm= confusion_matrix(y_test, y_pred_ada, labels= ada.classes_)
print(classification_report(y_test, y_pred_ada, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_ada, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_ada))
print('AUC:', roc_auc_score(y_test, y_pred_prob_ada))

              precision    recall  f1-score   support

           0       0.94      0.92      0.93      8517
           1       0.37      0.45      0.41       912

    accuracy                           0.87      9429
   macro avg       0.65      0.68      0.67      9429
weighted avg       0.88      0.87      0.88      9429

Precision: 0.36752899197145406
Accuracy: 0.8717785555202037
AUC: 0.8282857659294414


In [27]:
param_grid= {'n_estimators': [50, 100, 200]}

gs_ada= GridSearchCV(estimator= ada, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_ada.fit(X_sm_tl_selected, y_sm_tl)
print("Best Parameters:", gs_ada.best_params_)
print("Best Precision Score:", gs_ada.best_score_)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'n_estimators': 200}
Best Precision Score: 0.9463871046426153


In [28]:
tuned_ada= gs_ada.best_estimator_
y_pred_tuned_ada= tuned_ada.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_ada= tuned_ada.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_ada, labels= tuned_ada.classes_)
print(classification_report(y_test, y_pred_tuned_ada, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_ada, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_ada))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_ada))

              precision    recall  f1-score   support

           0       0.93      0.95      0.94      8517
           1       0.39      0.29      0.33       912

    accuracy                           0.89      9429
   macro avg       0.66      0.62      0.64      9429
weighted avg       0.87      0.89      0.88      9429

Precision: 0.39060205580029367
Accuracy: 0.887474811750981
AUC: 0.8277064936175121


--- GradientBoosting ---

In [29]:
from sklearn.ensemble import GradientBoostingClassifier
grb= GradientBoostingClassifier(random_state=15)
grb.fit(X_sm_tl_selected, y_sm_tl)
y_pred_grb= grb.predict(X_sm_tl_test_selected)
y_pred_prob_grb= grb.predict_proba(X_sm_tl_test_selected)[:,1]


cm= confusion_matrix(y_test, y_pred_grb, labels= grb.classes_)
print(classification_report(y_test, y_pred_grb, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_grb, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_grb))
print('AUC:', roc_auc_score(y_test, y_pred_prob_grb))

              precision    recall  f1-score   support

           0       0.93      0.95      0.94      8517
           1       0.38      0.30      0.34       912

    accuracy                           0.89      9429
   macro avg       0.65      0.62      0.64      9429
weighted avg       0.87      0.89      0.88      9429

Precision: 0.3814147018030513
Accuracy: 0.885141584473433
AUC: 0.8238705123293146


In [30]:
param_grid= {'learning_rate': [0.01, 0.1, 0.2]}

gs_grb= GridSearchCV(estimator= grb, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_grb.fit(X_sm_tl_selected, y_sm_tl)

print("Best Parameters:", gs_grb.best_params_)
print("Best Precision Score:", gs_grb.best_score_)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'learning_rate': 0.2}
Best Precision Score: 0.9670800563549322


In [31]:
tuned_grb= gs_grb.best_estimator_
y_pred_tuned_grb= tuned_grb.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_grb= tuned_grb.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_grb, labels= tuned_grb.classes_)
print(classification_report(y_test, y_pred_tuned_grb, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_grb, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_grb))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_grb))

              precision    recall  f1-score   support

           0       0.92      0.97      0.94      8517
           1       0.43      0.20      0.27       912

    accuracy                           0.90      9429
   macro avg       0.67      0.59      0.61      9429
weighted avg       0.87      0.90      0.88      9429

Precision: 0.43058823529411766
Accuracy: 0.8970198324318591
AUC: 0.8194732825370931


--- XGB ---

In [32]:
from xgboost import XGBClassifier
xgb= XGBClassifier(random_state=15)
xgb.fit(X_sm_tl_selected, y_sm_tl)
y_pred_xgb= xgb.predict(X_sm_tl_test_selected)
y_pred_prob_xgb= xgb.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_xgb, labels= xgb.classes_)
print(classification_report(y_test, y_pred_xgb, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_xgb, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_xgb))
print('AUC:', roc_auc_score(y_test, y_pred_prob_xgb))

              precision    recall  f1-score   support

           0       0.92      0.98      0.95      8517
           1       0.43      0.16      0.23       912

    accuracy                           0.90      9429
   macro avg       0.67      0.57      0.59      9429
weighted avg       0.87      0.90      0.88      9429

Precision: 0.4316109422492401
Accuracy: 0.8985046134266624
AUC: 0.8053316741130742


In [33]:
params_XGBoost= {'learning_rate': [0.01, 0.1, 1.0]}

gs_xgb= GridSearchCV(estimator= xgb, param_grid= params_XGBoost, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_xgb.fit(X_sm_tl_selected, y_sm_tl)

print("Best Parameters:", gs_xgb.best_params_)
print("Best Precision Score:", gs_xgb.best_score_)

tuned_xgb= gs_xgb.best_estimator_
y_pred_tuned_xgb= tuned_xgb.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_xgb= tuned_xgb.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_xgb, labels= tuned_xgb.classes_)
print(classification_report(y_test, y_pred_tuned_xgb, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_xgb, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_xgb))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_xgb))

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'learning_rate': 0.1}
Best Precision Score: 0.9678915039680642
              precision    recall  f1-score   support

           0       0.92      0.97      0.94      8517
           1       0.42      0.21      0.28       912

    accuracy                           0.90      9429
   macro avg       0.67      0.59      0.61      9429
weighted avg       0.87      0.90      0.88      9429

Precision: 0.42410714285714285
Accuracy: 0.8960653303637713
AUC: 0.8146020909677033


--- LGBM ---

In [34]:
from lightgbm import LGBMClassifier
lgm= LGBMClassifier(random_state=15)
lgm.fit(X_sm_tl_selected, y_sm_tl)
y_pred_lgm= lgm.predict(X_sm_tl_test_selected)
y_pred_prob_lgm= lgm.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_lgm, labels= lgm.classes_)
print(classification_report(y_test, y_pred_lgm, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_lgm, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_lgm))
print('AUC:', roc_auc_score(y_test, y_pred_prob_lgm))

[LightGBM] [Info] Number of positive: 33993, number of negative: 33993
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002363 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2550
[LightGBM] [Info] Number of data points in the train set: 67986, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
              precision    recall  f1-score   support

           0       0.92      0.98      0.95      8517
           1       0.44      0.16      0.23       912

    accuracy                           0.90      9429
   macro avg       0.68      0.57      0.59      9429
weighted avg       0.87      0.90      0.88      9429

Precision: 0.4386503067484663
Accuracy: 0.8990348923533779
AUC: 0.8114244936339909


In [35]:
params_LGB= {'learning_rate': [0.001, 0.01, 0.1, 1.0],
             'num_leaves': [31, 127],
             'reg_alpha': [0.1, 0.5],
             'min_data_in_leaf': [30, 50, 100, 300, 400]}

gs_lgm= GridSearchCV(estimator= lgm, param_grid= params_LGB, cv=5, scoring='precision', n_jobs=-1, verbose=2)
gs_lgm.fit(X_sm_tl_selected, y_sm_tl)

print("Best Parameters:", gs_lgm.best_params_)
print("Best Precision Score:", gs_lgm.best_score_)

tuned_lgm= gs_lgm.best_estimator_
y_pred_tuned_lgm= tuned_lgm.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_lgm= tuned_lgm.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_lgm, labels= tuned_lgm.classes_)
print(classification_report(y_test, y_pred_tuned_lgm, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_lgm, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_lgm))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_lgm))

Fitting 5 folds for each of 80 candidates, totalling 400 fits
[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[LightGBM] [Info] Number of positive: 33993, number of negative: 33993
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004733 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2550
[LightGBM] [Info] Number of data points in the train set: 67986, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
Best Parameters: {'learning_rate': 0.1, 'min_data_in_leaf': 30, 'num_leaves': 31, 'reg_alpha': 0.5}
Best Precision Score: 0.9755018417146651
[LightGBM] [Warning] min_data_in_leaf is set=30, m

--- CatBoost ---

In [36]:
from catboost import CatBoostClassifier
cat= CatBoostClassifier(random_state=15)
cat.fit(X_sm_tl_selected, y_sm_tl)
y_pred_cat= cat.predict(X_sm_tl_test_selected)
y_pred_prob_cat= cat.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_cat, labels= cat.classes_)
print(classification_report(y_test, y_pred_cat, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_cat, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_cat))
print('AUC:', roc_auc_score(y_test, y_pred_prob_cat))

Learning rate set to 0.062428
0:	learn: 0.6240281	total: 153ms	remaining: 2m 32s
1:	learn: 0.5685555	total: 166ms	remaining: 1m 22s
2:	learn: 0.5237194	total: 181ms	remaining: 1m
3:	learn: 0.4952311	total: 195ms	remaining: 48.5s
4:	learn: 0.4694224	total: 208ms	remaining: 41.5s
5:	learn: 0.4447680	total: 221ms	remaining: 36.6s
6:	learn: 0.4244359	total: 234ms	remaining: 33.2s
7:	learn: 0.4067703	total: 246ms	remaining: 30.5s
8:	learn: 0.3873743	total: 258ms	remaining: 28.4s
9:	learn: 0.3783623	total: 270ms	remaining: 26.7s
10:	learn: 0.3675299	total: 283ms	remaining: 25.4s
11:	learn: 0.3576438	total: 295ms	remaining: 24.3s
12:	learn: 0.3438695	total: 308ms	remaining: 23.4s
13:	learn: 0.3341373	total: 320ms	remaining: 22.5s
14:	learn: 0.3235142	total: 332ms	remaining: 21.8s
15:	learn: 0.3147460	total: 344ms	remaining: 21.2s
16:	learn: 0.3097346	total: 357ms	remaining: 20.6s
17:	learn: 0.3034322	total: 369ms	remaining: 20.1s
18:	learn: 0.2968769	total: 381ms	remaining: 19.7s
19:	learn: 0

162:	learn: 0.1954010	total: 2.29s	remaining: 11.8s
163:	learn: 0.1953085	total: 2.3s	remaining: 11.7s
164:	learn: 0.1949402	total: 2.31s	remaining: 11.7s
165:	learn: 0.1948725	total: 2.33s	remaining: 11.7s
166:	learn: 0.1948390	total: 2.34s	remaining: 11.7s
167:	learn: 0.1945737	total: 2.35s	remaining: 11.7s
168:	learn: 0.1945383	total: 2.37s	remaining: 11.6s
169:	learn: 0.1943547	total: 2.38s	remaining: 11.6s
170:	learn: 0.1943059	total: 2.39s	remaining: 11.6s
171:	learn: 0.1939376	total: 2.41s	remaining: 11.6s
172:	learn: 0.1938678	total: 2.42s	remaining: 11.6s
173:	learn: 0.1938027	total: 2.43s	remaining: 11.6s
174:	learn: 0.1937468	total: 2.44s	remaining: 11.5s
175:	learn: 0.1937045	total: 2.46s	remaining: 11.5s
176:	learn: 0.1936144	total: 2.47s	remaining: 11.5s
177:	learn: 0.1935149	total: 2.49s	remaining: 11.5s
178:	learn: 0.1934535	total: 2.5s	remaining: 11.5s
179:	learn: 0.1931028	total: 2.51s	remaining: 11.4s
180:	learn: 0.1928756	total: 2.52s	remaining: 11.4s
181:	learn: 0.

322:	learn: 0.1787962	total: 4.48s	remaining: 9.38s
323:	learn: 0.1787608	total: 4.49s	remaining: 9.37s
324:	learn: 0.1786610	total: 4.5s	remaining: 9.35s
325:	learn: 0.1786253	total: 4.52s	remaining: 9.34s
326:	learn: 0.1785824	total: 4.53s	remaining: 9.32s
327:	learn: 0.1785588	total: 4.54s	remaining: 9.31s
328:	learn: 0.1785292	total: 4.56s	remaining: 9.29s
329:	learn: 0.1784794	total: 4.57s	remaining: 9.28s
330:	learn: 0.1784503	total: 4.58s	remaining: 9.26s
331:	learn: 0.1783737	total: 4.59s	remaining: 9.25s
332:	learn: 0.1783518	total: 4.61s	remaining: 9.23s
333:	learn: 0.1782971	total: 4.62s	remaining: 9.21s
334:	learn: 0.1781856	total: 4.64s	remaining: 9.21s
335:	learn: 0.1781674	total: 4.66s	remaining: 9.22s
336:	learn: 0.1781186	total: 4.68s	remaining: 9.21s
337:	learn: 0.1780797	total: 4.69s	remaining: 9.19s
338:	learn: 0.1780536	total: 4.7s	remaining: 9.17s
339:	learn: 0.1779790	total: 4.72s	remaining: 9.15s
340:	learn: 0.1779156	total: 4.73s	remaining: 9.14s
341:	learn: 0.

482:	learn: 0.1729246	total: 6.66s	remaining: 7.13s
483:	learn: 0.1728962	total: 6.68s	remaining: 7.12s
484:	learn: 0.1728523	total: 6.69s	remaining: 7.1s
485:	learn: 0.1728229	total: 6.7s	remaining: 7.09s
486:	learn: 0.1728035	total: 6.71s	remaining: 7.07s
487:	learn: 0.1727734	total: 6.73s	remaining: 7.06s
488:	learn: 0.1727416	total: 6.74s	remaining: 7.04s
489:	learn: 0.1727178	total: 6.75s	remaining: 7.03s
490:	learn: 0.1726913	total: 6.77s	remaining: 7.02s
491:	learn: 0.1726705	total: 6.78s	remaining: 7s
492:	learn: 0.1726323	total: 6.79s	remaining: 6.99s
493:	learn: 0.1726148	total: 6.81s	remaining: 6.97s
494:	learn: 0.1725864	total: 6.82s	remaining: 6.96s
495:	learn: 0.1725692	total: 6.83s	remaining: 6.94s
496:	learn: 0.1725497	total: 6.84s	remaining: 6.93s
497:	learn: 0.1725225	total: 6.86s	remaining: 6.92s
498:	learn: 0.1724956	total: 6.87s	remaining: 6.9s
499:	learn: 0.1724754	total: 6.89s	remaining: 6.89s
500:	learn: 0.1724390	total: 6.9s	remaining: 6.87s
501:	learn: 0.17240

645:	learn: 0.1693694	total: 8.94s	remaining: 4.9s
646:	learn: 0.1693496	total: 8.95s	remaining: 4.88s
647:	learn: 0.1693338	total: 8.96s	remaining: 4.87s
648:	learn: 0.1693038	total: 8.98s	remaining: 4.86s
649:	learn: 0.1692788	total: 8.99s	remaining: 4.84s
650:	learn: 0.1692596	total: 9s	remaining: 4.83s
651:	learn: 0.1692448	total: 9.02s	remaining: 4.82s
652:	learn: 0.1692228	total: 9.04s	remaining: 4.8s
653:	learn: 0.1691911	total: 9.05s	remaining: 4.79s
654:	learn: 0.1691810	total: 9.06s	remaining: 4.77s
655:	learn: 0.1691570	total: 9.08s	remaining: 4.76s
656:	learn: 0.1691315	total: 9.09s	remaining: 4.75s
657:	learn: 0.1691094	total: 9.1s	remaining: 4.73s
658:	learn: 0.1691008	total: 9.12s	remaining: 4.72s
659:	learn: 0.1690755	total: 9.13s	remaining: 4.7s
660:	learn: 0.1690670	total: 9.14s	remaining: 4.69s
661:	learn: 0.1690598	total: 9.16s	remaining: 4.68s
662:	learn: 0.1690366	total: 9.19s	remaining: 4.67s
663:	learn: 0.1690158	total: 9.2s	remaining: 4.65s
664:	learn: 0.168995

811:	learn: 0.1667958	total: 11.2s	remaining: 2.6s
812:	learn: 0.1667866	total: 11.2s	remaining: 2.59s
813:	learn: 0.1667758	total: 11.3s	remaining: 2.57s
814:	learn: 0.1667678	total: 11.3s	remaining: 2.56s
815:	learn: 0.1667589	total: 11.3s	remaining: 2.54s
816:	learn: 0.1667413	total: 11.3s	remaining: 2.53s
817:	learn: 0.1667277	total: 11.3s	remaining: 2.52s
818:	learn: 0.1666976	total: 11.3s	remaining: 2.5s
819:	learn: 0.1666699	total: 11.3s	remaining: 2.49s
820:	learn: 0.1666607	total: 11.4s	remaining: 2.48s
821:	learn: 0.1666372	total: 11.4s	remaining: 2.46s
822:	learn: 0.1666264	total: 11.4s	remaining: 2.45s
823:	learn: 0.1666153	total: 11.4s	remaining: 2.43s
824:	learn: 0.1666068	total: 11.4s	remaining: 2.42s
825:	learn: 0.1665895	total: 11.4s	remaining: 2.41s
826:	learn: 0.1665819	total: 11.4s	remaining: 2.39s
827:	learn: 0.1665759	total: 11.5s	remaining: 2.38s
828:	learn: 0.1665667	total: 11.5s	remaining: 2.36s
829:	learn: 0.1665507	total: 11.5s	remaining: 2.35s
830:	learn: 0.

970:	learn: 0.1649105	total: 13.4s	remaining: 401ms
971:	learn: 0.1648938	total: 13.4s	remaining: 387ms
972:	learn: 0.1648871	total: 13.4s	remaining: 373ms
973:	learn: 0.1648798	total: 13.5s	remaining: 359ms
974:	learn: 0.1648674	total: 13.5s	remaining: 345ms
975:	learn: 0.1648544	total: 13.5s	remaining: 332ms
976:	learn: 0.1648460	total: 13.5s	remaining: 318ms
977:	learn: 0.1648417	total: 13.5s	remaining: 304ms
978:	learn: 0.1648293	total: 13.5s	remaining: 290ms
979:	learn: 0.1648212	total: 13.5s	remaining: 276ms
980:	learn: 0.1648168	total: 13.6s	remaining: 262ms
981:	learn: 0.1648095	total: 13.6s	remaining: 249ms
982:	learn: 0.1647982	total: 13.6s	remaining: 235ms
983:	learn: 0.1647881	total: 13.6s	remaining: 221ms
984:	learn: 0.1647775	total: 13.6s	remaining: 207ms
985:	learn: 0.1647671	total: 13.6s	remaining: 193ms
986:	learn: 0.1647582	total: 13.6s	remaining: 180ms
987:	learn: 0.1647488	total: 13.6s	remaining: 166ms
988:	learn: 0.1647352	total: 13.7s	remaining: 152ms
989:	learn: 

In [37]:
params_CatBoost= {
                  'depth': [3,5,10],
                  'learning_rate' : [0.01,0.1,1],
                  'iterations' : [5,10,50,100]
                 }

gs_cat= GridSearchCV(estimator= cat, param_grid= params_CatBoost, cv=5, scoring='precision', n_jobs=-1, verbose=2)
gs_cat.fit(X_sm_tl_selected, y_sm_tl)

print("Best Parameters:", gs_cat.best_params_)
print("Best Precision Score:", gs_cat.best_score_)

tuned_cat= gs_cat.best_estimator_
y_pred_tuned_cat= tuned_cat.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_cat= tuned_cat.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_cat, labels= tuned_cat.classes_)
print(classification_report(y_test, y_pred_tuned_cat, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_cat, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_cat))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_cat))

Fitting 5 folds for each of 36 candidates, totalling 180 fits
0:	learn: 0.5008906	total: 17.3ms	remaining: 1.72s
1:	learn: 0.4310172	total: 67.2ms	remaining: 3.29s
2:	learn: 0.3920610	total: 99.5ms	remaining: 3.22s
3:	learn: 0.3688940	total: 125ms	remaining: 3s
4:	learn: 0.3419866	total: 179ms	remaining: 3.41s
5:	learn: 0.3154248	total: 205ms	remaining: 3.21s
6:	learn: 0.3014836	total: 226ms	remaining: 3.01s
7:	learn: 0.2818227	total: 249ms	remaining: 2.87s
8:	learn: 0.2743434	total: 266ms	remaining: 2.69s
9:	learn: 0.2589468	total: 295ms	remaining: 2.65s
10:	learn: 0.2490023	total: 330ms	remaining: 2.67s
11:	learn: 0.2415042	total: 369ms	remaining: 2.7s
12:	learn: 0.2347994	total: 416ms	remaining: 2.78s
13:	learn: 0.2308294	total: 448ms	remaining: 2.75s
14:	learn: 0.2297396	total: 469ms	remaining: 2.66s
15:	learn: 0.2252639	total: 518ms	remaining: 2.72s
16:	learn: 0.2215853	total: 538ms	remaining: 2.63s
17:	learn: 0.2190010	total: 590ms	remaining: 2.69s
18:	learn: 0.2179297	total: 614

--- Visualisation ---

In [38]:
precision_scores= {
                    'Logistic Regression Precision:': precision_score(y_test, y_pred_lr, zero_division= 0),
                    'Decision Tree Precision:': precision_score(y_test, y_pred_dt, zero_division= 0),
                    'Tuned Decision Tree Precision:': precision_score(y_test, y_pred_tuned_dt, zero_division= 0),
                    'KNeighborsClassifier Precision:': precision_score(y_test, y_pred_knn, zero_division= 0),
                    'Tuned KNeighborsClassifier Precision:': precision_score(y_test, y_pred_tuned_knn, zero_division= 0),
                    'GaussianNB Precision:': precision_score(y_test, y_pred_nb, zero_division= 0),
                    'SVM Precision:': precision_score(y_test, y_pred_svc, zero_division= 0),
                    'Random Forest Precision:': precision_score(y_test, y_pred_rf, zero_division= 0),
                    'Tuned Random Forest Precision:': precision_score(y_test, y_pred_tuned_rf, zero_division= 0),
                    'AdaBoost Precision:': precision_score(y_test, y_pred_ada, zero_division= 0),
                    'Tuned AdaBoost Precision:': precision_score(y_test, y_pred_tuned_ada, zero_division= 0),
                    'GradientBoosting Precision:': precision_score(y_test, y_pred_grb, zero_division= 0),
                    'Tuned GradientBoosting Precision:': precision_score(y_test, y_pred_tuned_grb, zero_division= 0),
                    'XGB Precision:': precision_score(y_test, y_pred_xgb, zero_division= 0),
                    'Tuned XGB Precision:': precision_score(y_test, y_pred_tuned_xgb, zero_division= 0),
                    'LGBM Precision:': precision_score(y_test, y_pred_lgm, zero_division= 0),
                    'Tuned LGBM Precision:': precision_score(y_test, y_pred_tuned_lgm, zero_division= 0),
                    'CatBoost Precision:': precision_score(y_test, y_pred_cat, zero_division= 0),
                    'Tuned CatBoost Precision:': precision_score(y_test, y_pred_tuned_cat, zero_division= 0)
                  }

annova_precision= pd.DataFrame(list(precision_scores.items()), columns= ['Model', 'Precision Score'])
annova_precision= annova_precision.sort_values(by= 'Precision Score', ascending=False)
print(annova_precision)

                                    Model  Precision Score
16                  Tuned LGBM Precision:         0.441176
15                        LGBM Precision:         0.438650
18              Tuned CatBoost Precision:         0.438066
17                    CatBoost Precision:         0.434251
13                         XGB Precision:         0.431611
12      Tuned GradientBoosting Precision:         0.430588
14                   Tuned XGB Precision:         0.424107
10              Tuned AdaBoost Precision:         0.390602
8          Tuned Random Forest Precision:         0.381857
11            GradientBoosting Precision:         0.381415
9                     AdaBoost Precision:         0.367529
7                Random Forest Precision:         0.360996
1                Decision Tree Precision:         0.356643
2          Tuned Decision Tree Precision:         0.356643
3         KNeighborsClassifier Precision:         0.328378
6                          SVM Precision:         0.2919

In [39]:
accuracy_scores= {
                    'Logistic Regression Accuracy:': accuracy_score(y_test, y_pred_lr),
                    'Decision Tree Accuracy:': accuracy_score(y_test, y_pred_dt),
                    'Tuned Decision Tree Accuracy:': accuracy_score(y_test, y_pred_tuned_dt),
                    'KNeighborsClassifier Accuracy:': accuracy_score(y_test, y_pred_knn),
                    'Tuned KNeighborsClassifier Accuracy:': accuracy_score(y_test, y_pred_tuned_knn),
                    'GaussianNB Accuracy:': accuracy_score(y_test, y_pred_nb),
                    'SVM Accuracy:': accuracy_score(y_test, y_pred_svc),
                    'Random Forest Accuracy:': accuracy_score(y_test, y_pred_rf),
                    'Tuned Random Forest Accuracy:': accuracy_score(y_test, y_pred_tuned_rf),
                    'AdaBoost Accuracy:': accuracy_score(y_test, y_pred_ada),
                    'Tuned AdaBoost Accuracy:': accuracy_score(y_test, y_pred_tuned_ada),
                    'GradientBoosting Accuracy:': accuracy_score(y_test, y_pred_grb),
                    'Tuned GradientBoosting Accuracy:': accuracy_score(y_test, y_pred_tuned_grb),
                    'XGB Accuracy:': accuracy_score(y_test, y_pred_xgb),
                    'Tuned XGB Accuracy:': accuracy_score(y_test, y_pred_tuned_xgb),
                    'LGBM Accuracy:': accuracy_score(y_test, y_pred_lgm),
                    'Tuned LGBM Accuracy:': accuracy_score(y_test, y_pred_tuned_lgm),
                    'CatBoost Accuracy:': accuracy_score(y_test, y_pred_cat),
                    'Tuned CatBoost Accuracy:': accuracy_score(y_test, y_pred_tuned_cat)
                  }

annova_accuracy= pd.DataFrame(list(accuracy_scores.items()), columns= ['Model', 'Accuracy Score'])
annova_accuracy= annova_accuracy.sort_values(by= 'Accuracy Score', ascending=False)
print(annova_accuracy)

                                   Model  Accuracy Score
16                  Tuned LGBM Accuracy:        0.899459
15                        LGBM Accuracy:        0.899035
18              Tuned CatBoost Accuracy:        0.898929
17                    CatBoost Accuracy:        0.898717
13                         XGB Accuracy:        0.898505
12      Tuned GradientBoosting Accuracy:        0.897020
14                   Tuned XGB Accuracy:        0.896065
8          Tuned Random Forest Accuracy:        0.891399
2          Tuned Decision Tree Accuracy:        0.890232
1                Decision Tree Accuracy:        0.890232
7                Random Forest Accuracy:        0.889066
10              Tuned AdaBoost Accuracy:        0.887475
11            GradientBoosting Accuracy:        0.885142
4   Tuned KNeighborsClassifier Accuracy:        0.884929
3         KNeighborsClassifier Accuracy:        0.876339
9                     AdaBoost Accuracy:        0.871779
6                          SVM 

In [40]:
auc_scores= {
                    'Logistic Regression AUC:': roc_auc_score(y_test, y_pred_prob_lr),
                    'Decision Tree AUC:': roc_auc_score(y_test, y_pred_prob_dt),
                    'Tuned Decision Tree AUC:': roc_auc_score(y_test, y_pred_prob_tuned_dt),
                    'KNeighborsClassifier AUC:': roc_auc_score(y_test, y_pred_prob_knn),
                    'Tuned KNeighborsClassifier AUC:': roc_auc_score(y_test, y_pred_prob_tuned_knn),
                    'GaussianNB AUC:': roc_auc_score(y_test, y_pred_prob_nb),
                    'SVM AUC:': roc_auc_score(y_test, y_pred_prob_svc),
                    'Random Forest AUC:': roc_auc_score(y_test, y_pred_prob_rf),
                    'Tuned Random Forest AUC:': roc_auc_score(y_test, y_pred_prob_tuned_rf),
                    'AdaBoost AUC:': roc_auc_score(y_test, y_pred_prob_ada),
                    'Tuned AdaBoost AUC:': roc_auc_score(y_test, y_pred_prob_tuned_ada),
                    'GradientBoosting AUC:': roc_auc_score(y_test, y_pred_prob_grb),
                    'Tuned GradientBoosting AUC:': roc_auc_score(y_test, y_pred_prob_tuned_grb),
                    'XGB AUC:': roc_auc_score(y_test, y_pred_prob_xgb),
                    'Tuned XGB AUC:': roc_auc_score(y_test, y_pred_prob_tuned_xgb),
                    'LGBM AUC:': roc_auc_score(y_test, y_pred_prob_lgm),
                    'Tuned LGBM AUC:': roc_auc_score(y_test, y_pred_prob_tuned_lgm),
                    'CatBoost AUC:': roc_auc_score(y_test, y_pred_prob_cat),
                    'Tuned CatBoost AUC:': roc_auc_score(y_test, y_pred_prob_tuned_cat)
                  }

annova_auc= pd.DataFrame(list(auc_scores.items()), columns= ['Model', 'AUC Score'])
annova_auc= annova_auc.sort_values(by= 'AUC Score', ascending=False)
print(annova_auc)

                              Model  AUC Score
0          Logistic Regression AUC:   0.830625
9                     AdaBoost AUC:   0.828286
10              Tuned AdaBoost AUC:   0.827706
11            GradientBoosting AUC:   0.823871
5                   GaussianNB AUC:   0.820906
12      Tuned GradientBoosting AUC:   0.819473
14                   Tuned XGB AUC:   0.814602
16                  Tuned LGBM AUC:   0.813311
18              Tuned CatBoost AUC:   0.811476
15                        LGBM AUC:   0.811424
17                    CatBoost AUC:   0.808364
13                         XGB AUC:   0.805332
8          Tuned Random Forest AUC:   0.788393
7                Random Forest AUC:   0.748194
6                          SVM AUC:   0.730684
3         KNeighborsClassifier AUC:   0.712552
1                Decision Tree AUC:   0.682299
2          Tuned Decision Tree AUC:   0.682299
4   Tuned KNeighborsClassifier AUC:   0.633734
